In [1]:
import pandas as pd

/Users/emilsataev/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/emilsataev/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [ ]:
df = pd.read_csv('notebooks_meta.csv')
kernels = pd.read_csv('Kernels.csv')
df_comps = pd.read_csv('df_new_filtered.csv')
df.head()

In [ ]:
kr_versions = pd.read_csv('KernelVersions.csv')
kernels.head()

In [4]:
df_submissions = pd.read_csv('Submissions.csv')


In [5]:
df_users = pd.read_csv('Users.csv')

In [ ]:
# Select only the relevant columns from df
df_relevant = df[['CurrentKernelVersionId', 'CompetitionId', 'UserId', 'PrivateLeaderboardRank']]

# Select only the relevant columns from kernels
kernels_relevant = kernels[['CurrentKernelVersionId', 'TotalComments', 'TotalVotes', 'CurrentUrlSlug']]

# Merge df with kernels to get kernel-related columns
merged_df = df_relevant.merge(kernels_relevant, on='CurrentKernelVersionId', how='left')

# Select only the relevant columns from df_comps
df_comps_relevant = df_comps[['Id', 'Title']]

# Merge the result with df_comps to get the competition name
merged_df = merged_df.merge(df_comps_relevant, left_on='CompetitionId', right_on='Id', how='left')

# Select only the relevant columns from df_submissions
df_submissions_relevant = df_submissions[['SourceKernelVersionId', 'PublicScoreFullPrecision']]

# Merge the result with df_submissions to get the kaggle_score
merged_df = merged_df.merge(df_submissions_relevant, left_on='CurrentKernelVersionId', right_on='SourceKernelVersionId', how='left')

# Select only the relevant columns from df_users
df_users_relevant = df_users[['Id', 'UserName']]

# Merge the result with df_users to get the UserName for kernel_link
merged_df = merged_df.merge(df_users_relevant, left_on='UserId', right_on='Id', how='left')

# Format kernel_link
merged_df['kernel_link'] = '/' + merged_df['UserName'] + '/' + merged_df['CurrentUrlSlug']

# Select and rename the required columns
result_df = merged_df[['CurrentKernelVersionId', 'TotalComments', 'TotalVotes', 'kernel_link', 'Title', 'PublicScoreFullPrecision', 'PrivateLeaderboardRank']]
result_df.columns = ['kernel_id', 'kaggle_comments', 'kaggle_upvotes', 'kernel_link', 'comp_name', 'kaggle_score', 'rank']

# Set kernel_id as the index
result_df.set_index('kernel_id', inplace=True)

result_df

In [51]:
result_df.to_csv('kernels_meta2.csv')

In [24]:
df_relevant = df[['CurrentKernelVersionId', 'code', 'CleanCode']]

df_relevant.columns = ['kernel_id', 'code', 'clean_code']

df_relevant.set_index('kernel_id', inplace=True)

df_relevant
df_relevant.to_csv('code_blocks2.csv')

In [38]:
import ast

In [47]:
# Function to create a new dataframe with code blocks
def create_code_blocks_df(df):
    data = []
    code_blocks_index = 0
    for index, row in df.iterrows():
        kernel_id = row['CurrentKernelVersionId']
        code_blocks = ast.literal_eval(row['code'])
        for code_block_id, code_block in enumerate(code_blocks):
            data.append({
                'code_blocks_index': code_blocks_index, 
                'kernel_id': kernel_id, 
                'code_block_id': code_block_id, 
                'code_block': code_block
            })
            code_blocks_index += 1
    
    result_df = pd.DataFrame(data)
    result_df.set_index('code_blocks_index', inplace=True)
    return result_df

code_blocks_df = create_code_blocks_df(df)

In [49]:
code_blocks_df.to_csv('code_blocks2.csv')